In [1]:
import pandas as pd

metasamples = pd.read_excel("../Metadata.xlsx")

In [2]:
metasamples.head()

,PATIENT_ID,SAMPLE_ID,SAMPLE_TYPE,GENDER,INCLUDED,DAYS_AFTER_TREATMENT,AGE_DX,VAR1,VAR2,VAR3,...,VAR5,VAR6,VAR7,LANE,FASTA1,FASTA2,ROUND,MSI_ISH,MSI_SCORE,TUMOR_ONLY
0,VHIO_01,1123_3_cfDNA,cfDNA,XX,Y,224.0,55.0,BEFORE,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/1123_1.fastq.gz,s3://scitron/exomes/1123_2.fastq.gz,FIRST,MSS,8.32,No
1,VHIO_01,1261_2_cfDNA,cfDNA,XX,Y,310.0,55.0,NaN,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/1261_1.fastq.gz,s3://scitron/exomes/1261_2.fastq.gz,FIRST,MSS,13.29,No
2,VHIO_01,B19_0970__VH_17_B_022863_A5__1_tumor,tumor,XX,Y,266.0,55.0,AFTER,F,PD,...,2L,1L_2L,triplete,1.0,s3://scitron/exomes/B19_0970_S3_R1_001.fastq.gz,s3://scitron/exomes/B19_0970_S3_R2_001.fastq.gz,FIRST,MSS,1.95,No
3,VHIO_01,B19_0971__17_22863_A1__0_gDNA,gDNA,XX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,s3://scitron/exomes/B19_0971_S4_R1_001.fastq.gz,s3://scitron/exomes/B19_0971_S4_R2_001.fastq.gz,FIRST,NaN,Na,Na
4,VHIO_02,1514_5_cfDNA,cfDNA,XY,Y,336.0,57.0,NaN,F,short SD,...,2L,1L_2L,doblete,1.0,s3://scitron/exomes/1514_1.fastq.gz,s3://scitron/exomes/1514_2.fastq.gz,FOURTH,MSS,7.57,No


In [3]:
from pathlib import Path

PATH = "/Users/jfnavarro/Projects/Scitron/Results/VariantCalling"

# Add tumor only and MSI columns 
tumor_only_values = list()
msi_scores = list()
for index, row in metasamples.iterrows():
    sample_id = row['SAMPLE_ID']
    sample_patient = row['PATIENT_ID']
    tumor_only = not any(metasamples[metasamples['PATIENT_ID'] == sample_patient]['SAMPLE_TYPE'] == 'gDNA')
    germline = row['SAMPLE_TYPE'] == 'gDNA'
    msi_value = 'Na'
    if germline:
        tumor_only_values.append('Na')
    elif tumor_only:
        tumor_only_values.append('Yes')
        for elem in Path(PATH).rglob('*{}_msisensor'.format(sample_id)):
            with open(elem, 'r') as filehandler:
                tokens = filehandler.readlines()[1]
                _,_,msi_value = tokens.split("\t")
    else:
        tumor_only_values.append('No')
        gdna = metasamples[(metasamples['PATIENT_ID'] == sample_patient)
                           & (metasamples['SAMPLE_TYPE'] == 'gDNA')]['SAMPLE_ID'].to_numpy()[0]
        for elem in Path(PATH).rglob('*{}_vs_{}_msisensor'.format(sample_id, gdna)):
            with open(elem, 'r') as filehandler:
                tokens = filehandler.readlines()[1]
                _,_,msi_value = tokens.split("\t")
    msi_scores.append(msi_value.strip())
metasamples['MSI_SCORE'] = msi_scores
metasamples['TUMOR_ONLY'] = tumor_only_values

In [4]:
metasamples.to_excel('../Metadata.xlsx', index=False)